# Monte Carlo

In [ ]:
from src import ForceField, LJBox, MonteCarlo

#### Define Lennard-Jones parameters

Have the constructor plot the LJ potential

In [ ]:
sigma = 1.0 
epsilon = 1.0 
cutoff = 2.5 

forcefield = ForceField(sigma=sigma, epsilon=epsilon, cutoff=cutoff)

#### Define system parameters

Have the constructor output the box dimensions.

In [ ]:
n_particles = 216 
number_density = 0.7

system = LJBox(n_particles, number_density)

#### Define Monte Carlo parameters

In [ ]:
temperature = 1.2
dx = 0.1            # Initial maximum displacement
target = 0.5        # Target acceptance probabality
n_relax = 10000     # Number of timesteps to relax from initial configuration
n_mc = 500000       # Total number of MC steps

mc = MonteCarlo(forcefield=forcefield,
                system=system,
                dx=dx,
                temperature=temperature,
                target=target)

#### Relax the system

In [ ]:
mc.relax(n_relax, adjust_freq=100)

In [ ]:
import nglview as nv
import mdtraj as md
traj = md.load('relax.xyz', top='system_init.gro')
nv.show_mdtraj(traj, representations=[{'type': 'spacefill', 'params': {'radius': 0.5}}])

#### Perform production Monte Carlo run

In [ ]:
mc.run(n_mc)

#### View the trajectory

#### Analyze the radial distribution function